# Données imbriquées et répétées
Apprenez à interroger des types de données complexes dans BigQuery.

#### Introduction
Jusqu'à présent, vous avez travaillé avec de nombreux types de données, y compris les **types numériques** (entiers, valeurs à virgule flottante), les **chaînes de caractères** et le **type DATETIME**. 
Dans ce tutoriel, vous apprendrez à interroger des données **imbriquées et répétées**. Ce sont les types de données les plus complexes que vous pouvez rencontrer dans les jeux de données BigQuery !

---
#### Données imbriquées
Considérons un jeu de données hypothétique contenant des informations sur les animaux de compagnie et leurs jouets. Nous pourrions organiser ces informations dans deux tables différentes (une table pour les animaux de compagnie et une table pour les jouets). La table des jouets pourrait contenir une colonne "Pet_ID" qui permettrait de faire correspondre chaque jouet à l'animal qui le possède.

Une autre option dans BigQuery est d'organiser toutes les informations dans une seule table, similaire à la table `pets_and_toys` ci-dessous.

**Données imbriquées**

Dans ce cas, toutes les informations de la table des jouets sont regroupées dans une seule colonne (la colonne "Toy" dans la table `pets_and_toys`). Nous appelons la colonne "Toy" dans la table `pets_and_toys` une colonne imbriquée, et nous disons que les champs "Name" et "Type" sont imbriqués à l'intérieur.

Les colonnes imbriquées ont le **type STRUCT** (ou **type RECORD**). Cela est reflété dans le schéma de la table ci-dessous.

Rappelons que nous appelons la structure d'une table son **schéma**. Si vous avez besoin de revoir comment interpréter un schéma de table, n'hésitez pas à consulter cette leçon du micro-cours d'introduction à SQL.

**Données imbriquées**

Pour interroger une colonne avec des données imbriquées, nous devons identifier chaque champ dans le contexte de la colonne qui le contient :

- `Toy.Name` fait référence au champ "Name" dans la colonne "Toy", et
- `Toy.Type` fait référence au champ "Type" dans la colonne "Toy".

**Données imbriquées**

Sinon, nos règles habituelles restent les mêmes - nous n'avons pas besoin de changer quoi que ce soit d'autre dans nos requêtes.

---
#### Données répétées
Considérons maintenant le cas (plus réaliste !) où chaque animal de compagnie peut avoir plusieurs jouets. Dans ce cas, pour regrouper ces informations dans une seule table, nous devons utiliser un type de données différent.

**Données répétées**

Nous disons que la colonne "Toys" contient des données répétées, car elle permet plusieurs valeurs pour chaque ligne. Cela est reflété dans le schéma de la table ci-dessous, où le mode de la colonne "Toys" apparaît comme '**REPEATED**'.

**Données répétées**

Chaque entrée dans un champ répété est un **ARRAY**, ou une liste ordonnée de (zéro ou plusieurs) valeurs du même type de données. Par exemple, l'entrée dans la colonne "Toys" pour Moon le Chien est [Frisbee, Bone, Rope], qui est un ARRAY avec trois valeurs.

Lors de l'interrogation de données répétées, nous devons mettre le nom de la colonne contenant les données répétées à l'intérieur d'une fonction `UNNEST()`.

**Données répétées**

Cela aplatit essentiellement les données répétées (qui sont ensuite ajoutées à la droite de la table) afin que nous ayons un élément par ligne. Pour une illustration de cela, consultez l'image ci-dessous.


---
#### Données imbriquées et répétées
Maintenant, que faire si les animaux de compagnie peuvent avoir plusieurs jouets, et que nous souhaitons garder une trace à la fois du nom et du type de chaque jouet ? Dans ce cas, nous pouvons rendre la colonne "Toys" à la fois imbriquée et répétée.

**Données répétées**

Dans la table `more_pets_and_toys` ci-dessus, "Name" et "Type" sont tous deux des champs contenus dans le STRUCT "Toys", et chaque entrée dans "Toys.Name" et "Toys.Type" est un ARRAY.

**Données répétées**

Regardons un exemple de requête.

**Données répétées**

Puisque la colonne "Toys" est répétée, nous l'aplatissons avec la fonction `UNNEST()`. Et, puisque nous donnons à la colonne aplatie un alias de `t`, nous pouvons faire référence aux champs "Name" et "Type" dans la colonne "Toys" comme `t.Name` et `t.Type`, respectivement.

Pour renforcer ce que vous avez appris, nous appliquerons ces idées à un jeu de données réel dans la section ci-dessous.

---
#### Exemple
Nous allons travailler avec le jeu de données **Google Analytics Sample**. Il contient des informations suivant le comportement des **visiteurs du magasin Google Merchandise**, un site e-commerce vendant des articles de marque Google.

Nous commençons par afficher les premières lignes de la table `ga_sessions_20170801`.  Cette table suit les visites sur le site web le **1er août 2017**.


In [7]:
from google.cloud import bigquery

# Créer un objet "Client"
client = bigquery.Client()

# Construire une référence au jeu de données "google_analytics_sample"
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construire une référence à la table "ga_sessions_20170801"
table_ref = dataset_ref.table("ga_sessions_20170801")

# Requête API - récupérer la table
table = client.get_table(table_ref)

# Aperçu des cinq premières lignes de la table
client.list_rows(table, max_results=5).to_dataframe()


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501591568,1501591568,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",3418334011779872055,None,None,Organic Search,Not Socially Engaged
1,<NA>,2,1501589647,1501589647,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2474397855041322408,None,None,Referral,Not Socially Engaged
2,<NA>,1,1501616621,1501616621,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",5870462820713110108,None,None,Referral,Not Socially Engaged
3,<NA>,1,1501601200,1501601200,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",9397809171349480379,None,None,Referral,Not Socially Engaged
4,<NA>,1,1501615525,1501615525,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6089902943184578335,None,None,Referral,Not Socially Engaged


Pour une description de chaque champ, référez-vous à ce dictionnaire de données.

La table a de nombreux champs imbriqués, ce que vous pouvez vérifier en regardant soit le dictionnaire de données (indice : recherchez les occurrences de 'RECORD' sur la page) soit l'aperçu de la table ci-dessus.

Dans notre première requête sur cette table, nous allons travailler avec les colonnes "**totals**" et "**device**".


In [10]:
print("Champ SCHEMA pour la colonne 'totals':\n")
print(table.schema[5])

Champ SCHEMA pour la colonne 'totals':

SchemaField('totals', 'RECORD', 'NULLABLE', None, None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('hits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('timeOnScreen', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('totalTransactionRevenue', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('sessionQ

In [12]:
print("\nChamp SCHEMA pour la colonne 'device':\n")
print(table.schema[7])


Champ SCHEMA pour la colonne 'device':

SchemaField('device', 'RECORD', 'NULLABLE', None, None, (SchemaField('browser', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('browserVersion', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('browserSize', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('operatingSystem', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('operatingSystemVersion', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('isMobile', 'BOOLEAN', 'NULLABLE', None, None, (), None), SchemaField('mobileDeviceBranding', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('mobileDeviceModel', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('mobileInputSelector', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('mobileDeviceInfo', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('mobileDeviceMarketingName', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('flashVersion', 'STRING', 'NULLABLE', None, 

---
Nous faisons référence au champ "**browser**" (qui est imbriqué dans la colonne "**device**") et au champ "**transactions**" (qui est imbriqué dans la colonne "**totals**") comme `device.browser` et `totals.transactions` dans la requête ci-dessous :

In [15]:
# Requête pour compter le nombre de transactions par navigateur
query = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """

# Exécuter la requête et retourner un DataFrame pandas
result = client.query(query).result().to_dataframe()
result.head()

,device_browser,total_transactions
0,Chrome,41
1,Safari,3
2,Firefox,1
3,Edge,<NA>
4,Coc Coc,<NA>


En stockant les informations dans les colonnes "**device**" et "**totals**" sous forme de STRUCTs (au lieu de tables séparées), nous évitons des **JOINs** coûteux. Cela augmente les performances et nous évite de nous soucier des clés de **JOIN** (et de savoir quelles tables contiennent exactement les données dont nous avons besoin).

---
Maintenant, nous allons travailler avec la colonne "**hits**" comme exemple de données à la fois imbriquées et répétées. Puisque :

- "**hits**" est un STRUCT (contient des données imbriquées) et est répété,
- "**hitNumber**", "**page**", et "**type**" sont tous imbriqués dans la colonne "**hits**", et
- "**pagePath**" est imbriqué dans le champ "**page**",

Nous pouvons interroger ces champs avec la syntaxe suivante :

In [18]:
# Requête pour déterminer le point d'entrée le plus populaire sur le site web
query = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Exécuter la requête et retourner un DataFrame pandas
result = client.query(query).result().to_dataframe()
result.head()

,path,counts
0,/home,1257
1,/google+redesign/shop+by+brand/youtube,587
2,/google+redesign/apparel/mens/mens+t+shirts,117
3,/signin.html,78
4,/basket.html,35


Dans ce cas, la plupart des utilisateurs arrivent sur le site via la page "/home".

#### Votre tour
Utilisez ce que vous avez appris pour interroger des types de données complexes dans un jeu de données réel.

Vous avez des questions ou des commentaires ? Visitez le forum de discussion du cours pour discuter avec d'autres apprenants.

---
# **Exercices**  
## Introduction  
Maintenant que vous savez comment interroger des données imbriquées et répétées, vous êtes prêt à tirer des informations intéressantes du dataset **GitHub Repos**.


### Question 1) Qui a réalisé le plus de commits en 2016 ?  
GitHub est la plateforme la plus populaire pour collaborer sur des projets logiciels. Un **dépôt GitHub** (*repository* ou *repo*) est une collection de fichiers associés à un projet spécifique, et un **commit GitHub** représente une modification qu'un utilisateur a apportée à un dépôt. Cet utilisateur est appelé un **committer**.

La table **sample_commits** contient un petit échantillon de commits GitHub, où chaque ligne correspond à un commit différent.  

La cellule de code ci-dessous récupère cette table et affiche les cinq premières lignes.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "sample_commits" table
table_ref = dataset_ref.table("sample_commits")

# API request - fetch the table
sample_commits_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(sample_commits_table, max_results=5).to_dataframe()

,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,afdba32e2a9ea729a9f9f280dbf6c718773c7ded,d77cca8a096e5320f3194d4a6ca1b4fef2dc9b99,[d65e55d4999b394e37ffe12543ecd2a17b7c44fc],"{'name': 'Jason Gunthorpe', 'email': 'a99b91d7...","{'name': 'Peter Huewe', 'email': '014f16385c5a...",tpm: Pull everything related to /dev/tpmX into...,tpm: Pull everything related to /dev/tpmX into...,"[{'key': 'Signed-off-by', 'value': 'Jason Gunt...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
1,eb846d9f147455e4e5e1863bfb5e31974bb69b7c,443efbb146c7824508be817923bab04c2185810e,[3af6b35261182ff185db1f0fd271254147e2663e],"{'name': 'Hannes Reinecke', 'email': 'b0d1e9e4...","{'name': 'Christoph Hellwig', 'email': '923f77...",scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,"[{'key': 'Signed-off-by', 'value': 'Hannes Rei...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
2,f8798ccbefc0e4ef7438c080b7ba0410738c8cfa,9133440693c02314f1f6f95629b3594ce24ad0f8,[261e767628bb5971b9032439818237cc8511ea94],"{'name': 'Yong Zhang', 'email': '34add0fe16a1f...","{'name': 'Florian Tobias Schandinat', 'email':...",video: irq: Remove IRQF_DISABLED,video: irq: Remove IRQF_DISABLED\n\nSince comm...,"[{'key': 'Signed-off-by', 'value': 'Yong Zhang...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
3,b83ae6d421435c6204150300f1c25bfbd39cd62b,99c6b661ab7de05c2fd49aa62624d2d6bf8abc69,[de1414a654e66b81b5348dbc5259ecf2fb61655e],"{'name': 'Christoph Hellwig', 'email': '923f77...","{'name': 'Jens Axboe', 'email': 'cd8c6775e60d6...",fs: remove mapping->backing_dev_info,fs: remove mapping->backing_dev_info\n\nNow th...,"[{'key': 'Signed-off-by', 'value': 'Christoph ...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None
4,aaabee8b7686dfe49f10289cb4b7a817b99e5dd9,7ccc6cf829a93d46daf484164a5466c91eca2efa,"[795e9364215dc98b1dea888ebae22383ecbbb92a, 2f2...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...",Merge branch 'wl12xx-next' into for-linville,Merge branch 'wl12xx-next' into for-linville\n...,"[{'key': 'Conflicts', 'value': '', 'email': No...","[{'old_mode': 33188, 'new_mode': 33188, 'old_p...",<NA>,torvalds/linux,None


Exécutez la cellule de code suivante pour afficher le schéma de la table.  

Le code `sample_commits_table.schema` permet d'imprimer des informations sur toutes les colonnes de la table `sample_commits`. Cela inclut les noms des colonnes, leurs types de données et d'autres métadonnées utiles.

In [5]:
# Print information on all the columns in the table
sample_commits_table.schema

[SchemaField('commit', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('tree', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('parent', 'STRING', 'REPEATED', None, None, (), None),
 SchemaField('author', 'RECORD', 'NULLABLE', None, None, (SchemaField('name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None)), None),
 SchemaField('committer', 'RECORD', 'NULLABLE', None, None, (SchemaField('name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, ()

---
**Écrivez une requête pour trouver les individus ayant effectué le plus de commits dans cette table en 2016.**  

Votre requête doit retourner une table avec deux colonnes :  

- **committer_name** : contient le nom de chaque individu ayant effectué un commit (en 2016) dans la table.  
- **num_commits** : indique le nombre de commits que chaque individu a réalisés dans la table (en 2016).  

Triez la table de manière à ce que les personnes ayant le plus de commits apparaissent en premier.  

REMARQUE : Vous pouvez trouver le nom de chaque contributeur et la date du commit dans la colonne `"committer"`, respectivement dans les champs enfants `"name"` et `"date"`.

In [10]:
max_commits_query = """
    SELECT committer.name AS committer_name, 
           COUNT(*) AS num_commits
    FROM `bigquery-public-data.github_repos.sample_commits`
    WHERE EXTRACT(YEAR FROM committer.date) = 2016
    GROUP BY committer_name
    ORDER BY num_commits DESC
"""

# Exécuter la requête et renvoyer un DataFrame pandas
max_commits_result = client.query(max_commits_query).result().to_dataframe()
max_commits_result.head()

,committer_name,num_commits
0,Greg Kroah-Hartman,3545
1,David S. Miller,3120
2,TensorFlower Gardener,2449
3,Linus Torvalds,2424
4,Benjamin Pasero,1127


---

### Question 2) Regardons les langages !  

Tu vas maintenant travailler avec la table `languages`. Exécute la cellule de code ci-dessous pour afficher les premières lignes. 

In [13]:
# Construction d'une référence à la table "languages"
table_ref = dataset_ref.table("languages")

# Requête API - récupération de la table
languages_table = client.get_table(table_ref)

# Aperçu des cinq premières lignes de la table
client.list_rows(languages_table, max_results=5).to_dataframe()

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


Chaque ligne de la table `languages` correspond à un dépôt différent :  
- La colonne **`repo_name`** contient le nom du dépôt.  
- Le champ **`name`** dans la colonne **`language`** contient les langages de programmation utilisés dans le dépôt.  
- Le champ **`bytes`** dans la colonne **`language`** indique la taille des fichiers (en octets) pour le langage correspondant.  

Exécute la cellule suivante pour afficher le schéma de la table :

In [16]:
# Affichage des informations sur toutes les colonnes de la table
languages_table.schema

[SchemaField('repo_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('language', 'RECORD', 'REPEATED', None, None, (SchemaField('name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('bytes', 'INTEGER', 'NULLABLE', None, None, (), None)), None)]



#### Question sur le `sample_languages`  
On suppose maintenant que tu as accès à une table appelée `sample_languages`, qui contient seulement un petit sous-ensemble des lignes de la table `languages`. Cette table contient en fait seulement **trois lignes** !  

**Question :**  
Combien de lignes y aura-t-il dans le tableau retourné par la requête ci-dessous ?  

(⚠️ Une image est mentionnée ici, mais elle n'est pas affichée.)  

**Remplis ta réponse dans la cellule de code suivante.**

```
from google.cloud import bigquery

# Créer un client BigQuery
client = bigquery.Client()

# Requête pour extraire les langages de programmation
query = """
SELECT repo_name, language.name, language.bytes 
FROM `bigquery-public-data.github_repos.sample_languages`, 
UNNEST(languages) AS language
LIMIT 5
"""
```

---
### Question 3) Quel est le langage de programmation le plus populaire ?

Écris une requête exploitant les informations de la table `languages` pour déterminer quels langages de programmation apparaissent dans le plus grand nombre de dépôts.  

La table retournée par ta requête doit contenir deux colonnes :  

- **language_name** : le nom du langage de programmation  
- **num_repos** : le nombre de dépôts dans la table `languages` qui utilisent ce langage  

Trie la table de manière à ce que les langages apparaissant dans le plus grand nombre de dépôts soient affichés en premier.

--- 
### Question 4) Quels langages sont utilisés dans le dépôt avec le plus de langages ? 
Pour cette question, vous allez vous concentrer sur le dépôt dont le nom est 'polyrabbit/polyglot'.

Écrivez une requête qui renvoie une table avec une ligne pour chaque langage de ce dépôt. La table doit comporter deux colonnes :  

- `name` : le nom du langage de programmation  
- `bytes` : le nombre total de bytes de ce langage de programmation  
Classez la table par la colonne `bytes` de manière à ce que les langages de programmation occupant le plus d'espace dans le dépôt apparaissent en premier.